# Generate and Submit Job Scripts
This note is used to generate job scripts and submit generated jobs.

## Generate Job Scripts
### Environments

In [1]:
# Environment
# Box2D
Box2D_continuous_env_set = ['BipedalWalker-v2', 'BipedalWalkerHardcore-v2', 'LunarLanderContinuous-v2']
Box2D_discrete_env_set = ['LunarLander-v2']
# Classic control
Classic_continuous_env_set = ['Acrobot-v1', 'CartPole-v1', 'MountainCar-v0']
Classic_discrete_env_set = ['MountainCarContinuous-v0', 'Pendulum-v0']
# TODO: MuJoCo
MuJoCo_continuous_env_set = []
MuJoCo_discrete_env_set = []


### Different parameters are investigated:
* random seed
* exploration strategy
* target network update method
* double DDPG

In [1]:
import os

# Constant
JOB_GPU = 0
JOB_CPU = 1
JOB_MEMORY = '8000M'
PROCESS_SCRIPT = './ddpg_discrete_action.py'
SUMMARY_DIR = '../Double_DDPG_Results_no_monitor' # directory save job output
JOB_SCRIPT_DIR = './job_scripts'
USE_GYM_MONITOR = False

# Investigated Parameters
action_space_types = ['continuous', 'discrete']
environments = {'continuous':['BipedalWalker-v2', 'BipedalWalkerHardcore-v2', 'LunarLanderContinuous-v2',
                              'MountainCarContinuous-v0', 'Pendulum-v0',
                              'RoboschoolInvertedPendulum-v1', 'RoboschoolInvertedPendulumSwingup-v1',
                              'RoboschoolInvertedDoublePendulum-v1','RoboschoolReacher-v1',
                              'RoboschoolHopper-v1','RoboschoolWalker2d-v1',
                              'RoboschoolHalfCheetah-v1','RoboschoolAnt-v1', 
                              'RoboschoolHumanoid-v1', 'RoboschoolHumanoidFlagrun-v1', 
                              'RoboschoolHumanoidFlagrunHarder-v1', 'RoboschoolAtlasForwardWalk-v1',
                              'RoboschoolPong-v1'],
                'discrete':['LunarLander-v2',
                            'Acrobot-v1', 'CartPole-v1', 'MountainCar-v0']}
# time formats "hours:minutes:seconds"
job_times = {'continuous':['23:00:00', '23:00:00', '23:00:00',
                           '24:00:00', '10:00:00',
                           '23:00:00', '23:00:00', 
                           '23:00:00', '23:00:00', 
                           '23:00:00', '23:00:00',
                           '23:00:00', '23:00:00',
                           '23:00:00', '23:00:00',
                           '23:00:00', '23:00:00',
                           '23:00:00'],
            'discrete':['23:00:00',
                        '10:00:00', '10:00:00', '10:00:00']}
# Continuous tasks:
# 1. BipedalWalker-v2        : 2019, Feb. 14, 23:30 Cedar (done)
# 2. BipedalWalkerHardcore-v2: 2019, Feb. 17, 21:45 Cedar (submitted)
# 3. LunarLanderContinuous-v2: 2019, Feb. 18, 01:35 Graham (submitted)
# 4. MountainCarContinuous-v0: 2019, Feb. 17, 20:05 Graham (submitted)
# 5. Pendulum-v0             : 2019, Feb. 17, 20:05 Graham(submitted)
# 6. RoboschoolAnt-v1        : 2019, Feb. 19, 01:10 Cedar(submitted)

# Discrete tasks:
# 1. LunarLander-v2          : 2019, Feb. 17, 01:26
# 2. Acrobot-v1              : 2019, Feb. 17, 01:05 
# 3. CartPole-v1             : 2019, Feb. 17, 01:15
# 4. MountainCar-v0          : 2019, Feb. 17, 01:21

# environments = {'continuous':[],
#                 'discrete':['Pendulum-v0']}

ddpg_methods = ['ddpg', 'doule_ddpg']
target_net_update_method = ['softcopy', 'hardcopy']
exploration_strategies = ['action_noise', 'epsilon_greedy']
random_seeds = [1, 2, 3, 4, 5]
run_num = 10

job_count = 0
# Create Job Scripts:
# 1. Action space type
for action_space_type in action_space_types:
    if action_space_type == 'continuous':
        action_space_type_flag = '--continuous-act-space-flag'
    elif action_space_type == 'discrete':
        action_space_type_flag = ''
    else:
        print('Wrong action space type!')
    # 2. Environments
    for index, env in enumerate(environments[action_space_type]):
        jobb_time_tmp = job_times[action_space_type][index]
        # 3. DDPG method: ddpg or double-ddpg
        for ddpg_name in ddpg_methods:
            if ddpg_name == 'ddpg':
                ddpg_methods_flag = '--double-ddpg-flag'
            elif ddpg_name == 'doule_ddpg':
                ddpg_methods_flag = ''
            else:
                print('Wrong ddpg method!')
            # 4. Target net update methods
            for target_net_update in target_net_update_method:
                if target_net_update == 'softcopy':
                    target_hard_copy_flag = ''
                elif target_net_update == 'hardcopy':
                    target_hard_copy_flag = '--target-hard-copy-flag'
                else:
                    print('Wrong target_net_update!')
                # 5. Exploration strategies    
                for exploration in exploration_strategies:
                    # 6. Seeds
                    for seed in random_seeds:
                        # 7. Runs
                        for run in range(run_num):
                            # Job Name Example: continuous_env_ddpg_softcopy_action_noise_seed1_run1.sh
                            job_name = '{0}_{1}_{2}_{3}_{4}_seed{5}_run{6}'.format(action_space_type,
                                                                                   env, 
                                                                                   ddpg_name,
                                                                                   target_net_update,
                                                                                   exploration,
                                                                                   seed,
                                                                                   run)
                            job_script_name = '{}.sh'.format(job_name)
                            # Summary Dir Example: ../Double_DDPG_Results/continuous/env/ddpg_softcopy_action_noise_seed1_run1
                            summary_dir = os.path.join(SUMMARY_DIR,
                                                       action_space_type, 
                                                       env,
                                                       '{0}_{1}_{2}_seed{3}_run{4}'.format(ddpg_name,
                                                                                           target_net_update,
                                                                                           exploration,
                                                                                           seed,
                                                                                           run))
                            # Job Script Example: ./job_scripts/continuous_env_ddpg_softcopy_action_noise_seed1_run1.sh
                            job_script_dir = os.path.join(JOB_SCRIPT_DIR, env)
                            if not os.path.exists(job_script_dir):
                                os.makedirs(job_script_dir)
                            with open(os.path.join(job_script_dir, job_script_name), 'w') as job_file:
                                # Write job script
                                job_file.write('#!/bin/bash\n')
                                job_file.write('#SBATCH --account=def-dkulic\n')
                                #job_file.write('#SBATCH --gres=gpu:{}       # request GPU generic resource\n'.format(JOB_GPU))
                                #job_file.write('#SBATCH --cpus-per-task={}  #Maximum of CPU cores per GPU request: 6 on Cedar, 16 on Graham.\n'.format(JOB_CPU))
                                job_file.write('#SBATCH --mem={}            # memory per node\n'.format(JOB_MEMORY))
                                job_file.write('#SBATCH --time={}           # time (DD-HH:MM)\n'.format(jobb_time_tmp))
                                job_output_f = os.path.abspath('{}/{}_%N-%j.out'.format('../Double_DDPG_Job_output', job_name))
                                job_file.write('#SBATCH --output={}  # %N for node name, %j for jobID\n'.format(job_output_f))
                                job_file.write('\n')
                                job_file.write('module load qt/5.9.6 python/3.6.3 nixpkgs/16.09  gcc/7.3.0 boost/1.68.0 cuda cudnn\n')
                                job_file.write('source ~/tf_gpu/bin/activate\n')
                                job_file.write('\n')
                                if USE_GYM_MONITOR:
                                    use_gym_monitor_flag = '--use-gym-monitor-flag'
                                else:
                                    use_gym_monitor_flag = ''
                                
                                # Example: python './ddpg_discrete_action.py' --env BipedalWalker-v2 --continuous-act-space-flag --random-seed 1 --exploration-strategy action_noise --use-gym-monitor-flag --summary-dir ../Double_DDPG_Results/continuous/env/ddpg_softcopy_action_noise_seed1_run1
                                main_command = "python {0} --env {1} --random-seed {2} --exploration-strategy {3} --summary-dir {4} {5} {6} {7} {8}\n".format(\
                                PROCESS_SCRIPT, 
                                env, 
                                seed, 
                                exploration, 
                                summary_dir, 
                                action_space_type_flag, 
                                ddpg_methods_flag, 
                                target_hard_copy_flag, 
                                use_gym_monitor_flag)
                                
                                if USE_GYM_MONITOR:
                                    job_file.write("{0} {1}\n".format("xvfb-run -a ", 
                                                                    main_command))
                                else:
                                    job_file.write("{0}\n".format(main_command))
                                job_count += 1
                                print('Job {}: {}'.format(job_count, job_script_name))
                                

Job 1: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run0.sh
Job 2: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run1.sh
Job 3: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run2.sh
Job 4: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run3.sh
Job 5: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run4.sh
Job 6: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run5.sh
Job 7: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run6.sh
Job 8: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run7.sh
Job 9: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run8.sh
Job 10: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run9.sh
Job 11: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run0.sh
Job 12: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run1.sh
Job 13: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run2.sh
Job 14: 

Job 123: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run2.sh
Job 124: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run3.sh
Job 125: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run4.sh
Job 126: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run5.sh
Job 127: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run6.sh
Job 128: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run7.sh
Job 129: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run8.sh
Job 130: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run9.sh
Job 131: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed4_run0.sh
Job 132: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed4_run1.sh
Job 133: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed4_run2.sh
Job 134: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed4_run3.sh
Job 135: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_

Job 243: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run2.sh
Job 244: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run3.sh
Job 245: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run4.sh
Job 246: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run5.sh
Job 247: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run6.sh
Job 248: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run7.sh
Job 249: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run8.sh
Job 250: continuous_BipedalWalker-v2_doule_ddpg_softcopy_action_noise_seed5_run9.sh
Job 251: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed1_run0.sh
Job 252: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed1_run1.sh
Job 253: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed1_run2.sh
Job 254: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greed

Job 366: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed2_run5.sh
Job 367: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed2_run6.sh
Job 368: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed2_run7.sh
Job 369: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed2_run8.sh
Job 370: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed2_run9.sh
Job 371: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run0.sh
Job 372: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run1.sh
Job 373: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run2.sh
Job 374: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run3.sh
Job 375: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run4.sh
Job 376: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed3_run5.sh
Job 377: continuous_BipedalWalker-v2_doule_ddpg_hardco

Job 487: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed4_run6.sh
Job 488: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed4_run7.sh
Job 489: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed4_run8.sh
Job 490: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed4_run9.sh
Job 491: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run0.sh
Job 492: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run1.sh
Job 493: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run2.sh
Job 494: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run3.sh
Job 495: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run4.sh
Job 496: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run5.sh
Job 497: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run6.sh
Job 498: continuous_BipedalWalke

...

Job 8199: discrete_CartPole-v1_ddpg_hardcopy_epsilon_greedy_seed5_run8.sh
Job 8200: discrete_CartPole-v1_ddpg_hardcopy_epsilon_greedy_seed5_run9.sh
Job 8201: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run0.sh
Job 8202: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run1.sh
Job 8203: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run2.sh
Job 8204: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run3.sh
Job 8205: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run4.sh
Job 8206: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run5.sh
Job 8207: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run6.sh
Job 8208: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run7.sh
Job 8209: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run8.sh
Job 8210: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed1_run9.sh
Job 8211: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed2_ru

Job 8315: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run4.sh
Job 8316: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run5.sh
Job 8317: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run6.sh
Job 8318: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run7.sh
Job 8319: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run8.sh
Job 8320: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed2_run9.sh
Job 8321: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run0.sh
Job 8322: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run1.sh
Job 8323: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run2.sh
Job 8324: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run3.sh
Job 8325: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run4.sh
Job 8326: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run5.sh
Job 8327: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_

Job 8457: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run6.sh
Job 8458: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run7.sh
Job 8459: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run8.sh
Job 8460: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run9.sh
Job 8461: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run0.sh
Job 8462: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run1.sh
Job 8463: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run2.sh
Job 8464: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run3.sh
Job 8465: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run4.sh
Job 8466: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run5.sh
Job 8467: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run6.sh
Job 8468: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run7.sh
Job 8469: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run8.sh

Job 8589: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh
Job 8590: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run9.sh
Job 8591: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run0.sh
Job 8592: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh
Job 8593: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run2.sh
Job 8594: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run3.sh
Job 8595: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run4.sh
Job 8596: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run5.sh
Job 8597: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run6.sh
Job 8598: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run7.sh
Job 8599: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run8.sh
Job 8600: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed5_run9.sh
Job 8601: discrete_MountainCar-v0_doule_ddpg_softcopy_action_noise_seed1_run

Job 8709: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run8.sh
Job 8710: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run9.sh
Job 8711: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run0.sh
Job 8712: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run1.sh
Job 8713: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run2.sh
Job 8714: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run3.sh
Job 8715: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run4.sh
Job 8716: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run5.sh
Job 8717: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run6.sh
Job 8718: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run7.sh
Job 8719: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run8.sh
Job 8720: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed2_run9.sh
Job 8721: discrete_MountainC

## Submit Job Scripts

In [ ]:
import os
JOB_SCRIPT_DIR = './job_scripts'
job_scripts_dir = os.path.join(JOB_SCRIPT_DIR, 'RoboschoolAnt-v1')
job_scripts_list = os.listdir(job_scripts_dir)

for job in job_scripts_list:
    print('Submitting job: {}'.format(job))
    code = os.system('sbatch {}'.format(os.path.join(job_scripts_dir, job)))
    print('{}: {}'.format(job, code))
    

Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh
continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed1_run2.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed1_run2.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed2_run0.sh
continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed2_run0.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed3_run2.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed3_run2.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh
continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh: 0
Submitting job: co

continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed4_run0.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed1_run6.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed1_run6.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed2_run9.sh
continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed2_run9.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed2_run0.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed2_run0.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed5_run8.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed5_run8.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed4_run2.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed4_run2.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_action_noise_

continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed5_run9.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed4_run6.sh
continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed4_run6.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed3_run6.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed3_run6.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_hardcopy_action_noise_seed5_run9.sh
continuous_RoboschoolAnt-v1_ddpg_hardcopy_action_noise_seed5_run9.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed5_run1.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed5_run1.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed1_run3.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_action_noise_seed1_run3.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_epsilon_greedy_seed2_run6.sh
continuous_R

continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed5_run1.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed5_run2.sh
continuous_RoboschoolAnt-v1_doule_ddpg_hardcopy_action_noise_seed5_run2.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed1_run2.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed1_run2.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed4_run0.sh
continuous_RoboschoolAnt-v1_doule_ddpg_softcopy_epsilon_greedy_seed4_run0.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed3_run4.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed3_run4.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed4_run9.sh
continuous_RoboschoolAnt-v1_ddpg_softcopy_epsilon_greedy_seed4_run9.sh: 0
Submitting job: continuous_RoboschoolAnt-v1_ddpg_hardcopy_epsilon_greedy_seed5_run9.sh
continuo